In [ ]:
import sqlite3
import pandas as pd
def get_connection(db_name):
    return sqlite3.connect(db_name)

def get_data(sku=None):
    conn = get_connection("data/retail_analytics.db")
    sales_df = pd.read_sql_query("SELECT * FROM sales;", conn)
    campaigns_df = pd.read_sql_query("SELECT * FROM campaigns;", conn)
    inventory_df = pd.read_sql_query("SELECT * FROM products;", conn)
    conn.close()
    return sales_df, campaigns_df, inventory_df

In [24]:
conn=get_connection("data/retail_analytics.db")

In [25]:
# 2. Create a cursor object to execute SQL queries
cursor = conn.cursor()

# 3. Execute a query
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")  # list tables
tables = cursor.fetchall()
print("Tables:", tables)

Tables: [('draft_purchase_order',), ('demand_forecast_new',), ('out_of_stock',), ('stores',), ('suppliers',), ('customers',), ('products',), ('sales',), ('campaigns',), ('reorders',), ('supplier_skus',), ('demand_forecast',)]


In [27]:
# --- OPTIONAL: Load directly into Pandas DataFrame ---
df = pd.read_sql_query("SELECT * FROM demand_forecast;", conn)
print(df)

      sale_date   sku_id store_id  demand     lag_1  day_of_week  is_weekend  \
0    2025-08-10  SKU0001  STORE01    6.59  2.000000          6.0         1.0   
1    2025-08-11  SKU0001  STORE01    6.32  6.591347          0.0         0.0   
2    2025-08-12  SKU0001  STORE01    5.40  6.317193          1.0         0.0   
3    2025-08-13  SKU0001  STORE01    3.73  5.402296          2.0         0.0   
4    2025-08-14  SKU0001  STORE01    3.63  3.730859          3.0         0.0   
..          ...      ...      ...     ...       ...          ...         ...   
695  2025-08-12  SKU0100  STORE01    6.55  6.985379          1.0         0.0   
696  2025-08-13  SKU0100  STORE01    6.63  6.552753          2.0         0.0   
697  2025-08-14  SKU0100  STORE01    6.82  6.634023          3.0         0.0   
698  2025-08-15  SKU0100  STORE01    7.79  6.817474          4.0         0.0   
699  2025-08-16  SKU0100  STORE01    7.78  7.791848          5.0         1.0   

     month  discount_percent  success_r

In [28]:
df.columns

Index(['sale_date', 'sku_id', 'store_id', 'demand', 'lag_1', 'day_of_week',
       'is_weekend', 'month', 'discount_percent', 'success_rate',
       'propensity_score', 'date_added', 'days_predicted_for'],
      dtype='object')

In [ ]:
Present the forecast in a clear and structured format (e.g., table or chart if supported).

Summarize the total expected demand over 30 days.

Always show the Out of Stock in a neat tabular format show the total forecasted demand as well , show the oos Probability in percent

If applicable, highlight potential out-of-stock risks (based on demand vs. inventory threshold if available).

Fallbacks & Edge Cases:

If the user provides an invalid SKU or store, politely ask for clarification.

If data is missing, state that forecasts cannot be generated for those inputs.

In [ ]:
Role:
You are a demand forecasting and out-of-stock prediction agent.

Goal:
Provide forecasted demand values for a 30-day period based on the SKU(s) and store(s) mentioned in the user’s request.

Behavior & Rules:

Input Handling:

If no SKU and no store are provided → forecast demand for all SKUs across all stores.

If only SKU(s) is provided → forecast demand for the given SKU(s) across all stores.

If only store(s) is provided → forecast demand for all SKUs in that store(s).

If both SKU(s) and store(s) are provided → forecast demand for the given SKU(s) in the specified store(s).

Tool Usage:

Use the tool named forecast to retrieve the forecast values.

Always pass the following parameters to the tool:

start_date: today’s date (or the specified start date if user provides one).

sku_ids: list of SKU IDs (if applicable).

store_ids: list of Store IDs (if applicable).

Output Expectations
Always show the output in structured tabular format.
Present the forecast in a clear and structured format (e.g., table or chart if supported).
Summarize the total expected demand over 30 days.
Always show the Out of Stock in a neat tabular format show the total forecasted demand as well , show the oos Probability in percent


In [57]:
# out_of_stock_model.py
import pandas as pd
import numpy as np
from datetime import datetime
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    mean_squared_error, r2_score
)
from xgboost import XGBClassifier, XGBRegressor

def train_oos_models(forecast_df, inventory_df):
    """
    Train Out-of-Stock classification and regression models using forecasted demand.
    
    forecast_df: DataFrame with forecasted values per SKU/store/date
        Columns: 'sale_date', 'sku_id', 'store_id', 'demand', 'lag_1', 'day_of_week',
                 'is_weekend', 'month', 'discount_percent', 'success_rate',
                 'propensity_score', 'date_added', 'days_predicted_for'
    
    inventory_df: DataFrame with inventory information per SKU/store
        Columns: 'sku_id', 'store_id', 'current_stock', 'reorder_threshold',
                 'lead_time_days', 'cost_price', 'current_selling_price',
                 'last_restock_date', 'next_restock_date'
    
    Returns:
        clf: trained XGBClassifier (will OOS)
        reg: trained XGBRegressor (days to OOS)
    """

    # Copy forecast
    df = forecast_df.copy()
    df["forecast_date"] = pd.to_datetime(df["sale_date"])
    print(inventory_df)
    # Merge with inventory for stock info
    df = df.merge(
        inventory_df[[
            "sku_id", "store_id", "current_stock", "reorder_threshold", "cost_price", "current_selling_price",
            "last_restock_date", "next_restock_date"
        ]],
        on=["sku_id", "store_id"],
        how="left"
    )

    # Ensure datetime for restock dates
    df["last_restock_date"] = pd.to_datetime(df["last_restock_date"])
    df["next_restock_date"] = pd.to_datetime(df["next_restock_date"])

    # Feature engineering
    df["days_since_last_restock"] = (df["forecast_date"] - df["last_restock_date"]).dt.days
    df["days_until_next_restock"] = (df["next_restock_date"] - df["forecast_date"]).dt.days
    df["stock_ratio"] = df["current_stock"] / (df["reorder_threshold"] + 1)
    df["margin"] = df["current_selling_price"] - df["cost_price"]

    # Target for classification
    df["will_out_of_stock"] = (df["current_stock"] <= df["reorder_threshold"]).astype(int)

    # Target for regression (days to OOS)
    df["days_to_oos"] = np.where(
        df["demand"] > 0,
        (df["current_stock"] / df["demand"]).round().astype(int),
        np.nan
    )
    df_reg = df.dropna(subset=["days_to_oos"]).copy()

    # Features to use
    feature_cols = [
        "current_stock", "reorder_threshold",
        "demand", "lag_1", "day_of_week", "is_weekend", "month",
        "discount_percent", "success_rate", "propensity_score",
        "days_since_last_restock", "days_until_next_restock",
        "stock_ratio", "margin"
    ]

    X_cls = df[feature_cols]
    y_cls = df["will_out_of_stock"]

    X_reg = df_reg[feature_cols]
    y_reg = df_reg["days_to_oos"]

    # ===== SPLIT DATA =====
    Xc_train, Xc_test, yc_train, yc_test = train_test_split(
        X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
    )
    Xr_train, Xr_test, yr_train, yr_test = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )

    # ===== CLASSIFICATION MODEL =====
    clf = XGBClassifier(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        use_label_encoder=False,
        eval_metric="logloss",
        scale_pos_weight=(len(y_cls) - y_cls.sum()) / max(1, y_cls.sum())
    )
    clf.fit(Xc_train, yc_train)

    yc_pred = clf.predict(Xc_test)
    yc_prob = clf.predict_proba(Xc_test)[:, 1]

    print("\n📊 Classification Metrics:")
    print("Accuracy:", accuracy_score(yc_test, yc_pred))
    print("F1 Score:", f1_score(yc_test, yc_pred))
    print("ROC AUC:", roc_auc_score(yc_test, yc_prob))

    # ===== REGRESSION MODEL =====
    reg = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    reg.fit(Xr_train, yr_train)

    yr_pred = reg.predict(Xr_test)
    print("\n📊 Regression Metrics:")
    print("RMSE:", mean_squared_error(yr_test, yr_pred))
    print("R2 Score:", r2_score(yr_test, yr_pred))

    # ===== SAVE MODELS =====
    joblib.dump(clf, "out_of_stock_classifier.pkl")
    joblib.dump(reg, "days_to_oos_regressor.pkl")
    print("\n✅ Models saved: out_of_stock_classifier.pkl, days_to_oos_regressor.pkl")

    return clf, reg


In [51]:
conn=get_connection("data/retail_analytics.db")

In [131]:
inventory_df=pd.read_sql_query(f"SELECT * FROM reorders;", conn)

In [132]:
inventory_df.columns

Index(['reorder_id', 'sku_id', 'reorder_qty', 'reorder_date', 'status',
       'supplier_id', 'price_per_unit', 'reorder_fulfilment_date'],
      dtype='object')

In [133]:
inventory_df

,reorder_id,sku_id,reorder_qty,reorder_date,status,supplier_id,price_per_unit,reorder_fulfilment_date
0,REORDER00001,SKU0082,106,2025-08-25 00:00:00.000000,pending,S016,52.25,2025-08-25 00:00:00.000000
1,REORDER00002,SKU0015,47,2025-08-25 00:00:00.000000,cancelled,S015,97.23,None
2,REORDER00003,SKU0004,83,2025-08-25 00:00:00.000000,fulfilled,S001,40.47,2025-08-25 00:00:00.000000
3,REORDER00004,SKU0095,69,2025-08-25 00:00:00.000000,cancelled,S003,72.34,None
4,REORDER00005,SKU0036,68,2025-08-25 00:00:00.000000,pending,S010,68.49,2025-08-25 00:00:00.000000
...,...,...,...,...,...,...,...,...
295,REORDER00296,SKU0084,102,2025-08-25 00:00:00.000000,pending,S001,10.55,2025-08-25 00:00:00.000000
296,REORDER00297,SKU0083,139,2025-08-25 00:00:00.000000,cancelled,S005,88.29,None
297,REORDER00298,SKU0013,133,2025-08-25 00:00:00.000000,fulfilled,S014,60.97,2025-08-25 00:00:00.000000
298,REORDER00299,SKU0008,133,2025-08-25 00:00:00.000000,cancelled,S005,46.07,None


In [53]:
inventory_df,sales_df,campaigns_df=get_data()
inventory_df.columns


Index(['sale_id', 'customer_id', 'sku_id', 'quantity', 'sale_date', 'status',
       'fulfilled', 'amount', 'coupon', 'store_id'],
      dtype='object')

In [54]:
inventory_df.columns


Index(['sale_id', 'customer_id', 'sku_id', 'quantity', 'sale_date', 'status',
       'fulfilled', 'amount', 'coupon', 'store_id'],
      dtype='object')

In [58]:
clf,reg=train_oos_models(inventory_df=inventory_df,forecast_df=df)

     sku_id          product_name product_category  current_stock  \
0   SKU0001     Throw Significant             Home             70   
1   SKU0002           Glass Guess      Electronics            103   
2   SKU0003        Eight Describe            Books             30   
3   SKU0004            Notice Arm      Electronics             70   
4   SKU0005            What Whose            Books             75   
..      ...                   ...              ...            ...   
95  SKU0096            Point Lose             Home             70   
96  SKU0097    Yet Responsibility      Electronics             96   
97  SKU0098  Choice Environmental         Clothing            187   
98  SKU0099              End Play          Grocery            189   
99  SKU0100  Particular Recognize         Clothing            178   

    max_capacity  cost_price  current_selling_price store_id  \
0             70       30.25                  90.15  STORE01   
1            103       36.44               

/Users/ruhinshaikh/Desktop/Non-Pilot/retail_agentic/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [19:45:03] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 Regression Metrics:
RMSE: 1.1980857892736594
R2 Score: 0.987215687182324

✅ Models saved: out_of_stock_classifier.pkl, days_to_oos_regressor.pkl


In [127]:
# out_of_stock_model.py
import pandas as pd
import numpy as np
from datetime import datetime
import joblib, os
import matplotlib.pyplot as plt
import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, r2_score
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report

model_dir = "models_forecast"
os.makedirs(model_dir, exist_ok=True)

# ==================== TRAIN OOS MODELS USING FORECAST ====================
def train_oos_models(forecast_df, inventory_df):
    """
    Train Out-of-Stock classification and regression models using forecasted demand.
    """
    df = forecast_df.copy()
    df["forecast_date"] = pd.to_datetime(df["sale_date"])

    # Merge with inventory for stock info
    df = df.merge(
        inventory_df[[
            "sku_id", "store_id", "current_stock", "reorder_threshold",
            "cost_price", "current_selling_price", "last_restock_date", "next_restock_date"
        ]],
        on=["sku_id", "store_id"],
        how="left"
    )

    # Ensure datetime for restock dates
    df["last_restock_date"] = pd.to_datetime(df["last_restock_date"])
    df["next_restock_date"] = pd.to_datetime(df["next_restock_date"])

    # --- Feature Engineering ---
    df["days_since_last_restock"] = (df["forecast_date"] - df["last_restock_date"]).dt.days
    df["days_until_next_restock"] = (df["next_restock_date"] - df["forecast_date"]).dt.days
    df["stock_ratio"] = df["current_stock"] / (df["reorder_threshold"] + 1)
    df["margin"] = df["current_selling_price"] - df["cost_price"]

    # --- Classification target ---
    df["will_out_of_stock"] = (df["current_stock"] <= df["reorder_threshold"]).astype(int)

    # --- Regression target: days to OOS ---
    df["days_to_oos"] = np.where(
        df["demand"] > 0,
        (df["current_stock"] / df["demand"]).round().astype(int),
        np.nan
    )
    df_reg = df.dropna(subset=["days_to_oos"]).copy()

    # --- Feature columns ---
    feature_cols = [
        "current_stock", "reorder_threshold", "demand", "lag_1",
        "day_of_week", "is_weekend", "month", "discount_percent",
        "success_rate", "propensity_score", "days_since_last_restock",
        "days_until_next_restock", "stock_ratio", "margin"
    ]

    X_cls = df[feature_cols]
    y_cls = df["will_out_of_stock"]
    X_reg = df_reg[feature_cols]
    y_reg = df_reg["days_to_oos"]

    # ===== SPLIT DATA =====
    Xc_train, Xc_test, yc_train, yc_test = train_test_split(
        X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
    )
    Xr_train, Xr_test, yr_train, yr_test = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )

    # ===== CLASSIFICATION MODEL =====
    clf = XGBClassifier(
        n_estimators=300, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        use_label_encoder=False, eval_metric="logloss",
        scale_pos_weight=(len(y_cls)-y_cls.sum())/max(1, y_cls.sum())
    )
    clf.fit(Xc_train, yc_train)
    yc_pred = clf.predict(Xc_test)
    yc_prob = clf.predict_proba(Xc_test)[:, 1]

    print("\n📊 Classification Metrics:")
    print("Accuracy:", accuracy_score(yc_test, yc_pred))
    print("F1 Score:", f1_score(yc_test, yc_pred))
    print("ROC AUC:", roc_auc_score(yc_test, yc_prob))

    # ===== REGRESSION MODEL =====
    reg = XGBRegressor(
        n_estimators=300, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8, random_state=42
    )
    reg.fit(Xr_train, yr_train)
    yr_pred = reg.predict(Xr_test)

    print("\n📊 Regression Metrics:")
    print("RMSE:", mean_squared_error(yr_test, yr_pred))
    print("R2 Score:", r2_score(yr_test, yr_pred))

    # ===== SAVE MODELS =====
    joblib.dump(clf, os.path.join(model_dir, "oos_classifier_new.pkl"))
    joblib.dump(reg, os.path.join(model_dir, "days_to_oos_regressor.pkl"))
    print("\n✅ Models saved in", model_dir)

    return clf, reg

# ==================== PREDICTION FUNCTIONS ====================
def check_oos_with_forecast(inventory_df, forecast_df, horizon=7):
    """
    Merge forecast with inventory and compute OOS signals and days until OOS.
    """
    forecast_sum = (
        forecast_df
        .groupby(['sku_id', 'store_id'])['demand']
        .sum()
        .reset_index()
        .rename(columns={'demand': 'total_forecasted_demand'})
    )
    df = inventory_df.merge(forecast_sum, on=['sku_id', 'store_id'], how='left')
    df['total_forecasted_demand'] = df['total_forecasted_demand'].fillna(0)

    df['will_go_oos'] = df['current_stock'] < df['total_forecasted_demand']
    df['avg_daily_forecast'] = df['total_forecasted_demand'] / horizon

    def safe_days_until_oos(row):
        if row['avg_daily_forecast'] <= 0:
            return horizon
        days = row['current_stock'] / row['avg_daily_forecast']
        return int(days) if days > 0 and days != float('inf') else 0

    df['days_until_oos'] = df.apply(safe_days_until_oos, axis=1)
    return df
def predict_oos_and_days_until_oos(inventory_df, forecast_df, clf, reg, horizon=7, sku_ids=None, store_ids=None):
    """
    Predict Out-of-Stock and days until OOS using forecast + inventory.
    """
    # Copy forecast and ensure date
    df = forecast_df.copy()
    if 'sale_date' not in df.columns:
        raise KeyError("forecast_df must contain 'sale_date'")
    df['forecast_date'] = pd.to_datetime(df['sale_date'])

    # Merge inventory info
    df = df.merge(
        inventory_df[[
            'sku_id','store_id','current_stock','reorder_threshold',
            'cost_price','current_selling_price',
            'last_restock_date','next_restock_date'
        ]],
        on=['sku_id','store_id'], how='left'
    )

    # Ensure datetime for restock dates
    df['last_restock_date'] = pd.to_datetime(df['last_restock_date'])
    df['next_restock_date'] = pd.to_datetime(df['next_restock_date'])

    # Feature engineering
    df['days_since_last_restock'] = (df['forecast_date'] - df['last_restock_date']).dt.days
    df['days_until_next_restock'] = (df['next_restock_date'] - df['forecast_date']).dt.days
    df['stock_ratio'] = df['current_stock'] / (df['reorder_threshold'] + 1)
    df['margin'] = df['current_selling_price'] - df['cost_price']

    # Fill missing features with 0
    feature_cols = [
        'current_stock','reorder_threshold','demand','lag_1',
        'day_of_week','is_weekend','month','discount_percent',
        'success_rate','propensity_score','days_since_last_restock',
        'days_until_next_restock','stock_ratio','margin'
    ]
    for c in feature_cols:
        if c not in df.columns:
            df[c] = 0  # fallback for missing columns

    X = df[feature_cols].fillna(0)

    # Predict OOS classification
    df['predicted_oos'] = clf.predict(X)
    df['oos_probability'] = clf.predict_proba(X)[:,1]

    # Predict days until OOS
    df['predicted_days_until_oos'] = reg.predict(X).astype(int)
    df['predicted_days_until_oos'] = df['predicted_days_until_oos'].clip(lower=1, upper=horizon)

    # Filter by SKU/store if needed
    if sku_ids is not None:
        df = df[df['sku_id'].isin(sku_ids)]
    if store_ids is not None:
        df = df[df['store_id'].isin(store_ids)]

    return df[
        ['sku_id','store_id','current_stock','demand','predicted_oos','oos_probability','predicted_days_until_oos','forecast_date']
    ]

# def predict_oos_and_days_until_oos(inventory_df, forecast_df, clf, reg, horizon=7, sku_ids=None, store_ids=None):
#     # Merge forecast with inventory and compute OOS signals

#     df_oos = check_oos_with_forecast(inventory_df, forecast_df, horizon=horizon)
    
#     # Merge original forecast features for prediction
#     df = df_oos.merge(
#         forecast_df.drop(columns=['sale_date']), 
#         on=['sku_id','store_id'], 
#         how='left'
#     )
#     df['forecast_date'] = pd.to_datetime(df['sale_date'])
#     # Filter by SKU/store if provided
#     if sku_ids:
#         df = df[df['sku_id'].isin(sku_ids)]
#     if store_ids:
#         df = df[df['store_id'].isin(store_ids)]
#     df['forecast_date'] = pd.to_datetime(df['sale_date'])

#     df['days_since_last_restock'] = (pd.to_datetime(df['forecast_date']) - pd.to_datetime(df['last_restock_date'])).dt.days
#     df['days_until_next_restock'] = (pd.to_datetime(df['next_restock_date']) - pd.to_datetime(df['forecast_date'])).dt.days
#     df['stock_ratio'] = df['current_stock'] / (df['reorder_threshold'] + 1)
#     df['margin'] = df['current_selling_price'] - df['cost_price']


#     # Use only available columns for prediction
#     feature_cols = [c for c in [
#         'current_stock', 'reorder_threshold', 'demand', 'lag_1',
#         'day_of_week', 'is_weekend', 'month', 'discount_percent',
#         'success_rate', 'propensity_score', 'days_since_last_restock',
#         'days_until_next_restock', 'stock_ratio', 'margin',
#         'total_forecasted_demand', 'days_until_oos'
#     ] if c in df.columns]

#     X = df[feature_cols].fillna(0)

#     # Classification + Regression predictions
#     df['predicted_oos'] = clf.predict(X)
#     df['oos_probability'] = clf.predict_proba(X)[:,1]
#     df['predicted_days_until_oos'] = reg.predict(X)
#     df['predicted_days_until_oos'] = df['predicted_days_until_oos'].apply(lambda x: int(x) if x>0 else horizon)

#     return df[[
#         'sku_id', 'store_id', 'current_stock', 'total_forecasted_demand',
#         'predicted_oos', 'oos_probability', 'predicted_days_until_oos'
#     ]]


# def predict_oos_and_days_until_oos(inventory_df, forecast_df, clf, reg, horizon=7, sku_ids=None, store_ids=None):
#     df = check_oos_with_forecast(inventory_df, forecast_df, horizon=horizon)

#     # Filter by SKU/store if provided
#     if sku_ids:
#         df = df[df['sku_id'].isin(sku_ids)]
#     if store_ids:
#         df = df[df['store_id'].isin(store_ids)]

#     print(df.columns)
#    # --- Feature columns ---
#     feature_cols = [
#         "current_stock", "reorder_threshold", "demand", "lag_1",
#         "day_of_week", "is_weekend", "month", "discount_percent",
#         "success_rate", "propensity_score", "days_since_last_restock",
#         "days_until_next_restock", "stock_ratio", "margin"
#     ]


#     X = df[feature_cols].fillna(0)

#     # Classification + Regression predictions
#     df['predicted_oos'] = clf.predict(X)
#     df['oos_probability'] = clf.predict_proba(X)[:,1]
#     df['predicted_days_until_oos'] = reg.predict(X)
#     df['predicted_days_until_oos'] = df['predicted_days_until_oos'].apply(lambda x: int(x) if x>0 else horizon)

#     return df[[
#         'sku_id', 'store_id', 'current_stock', 'total_forecasted_demand',
#         'predicted_oos', 'oos_probability', 'predicted_days_until_oos'
#     ]]


In [121]:
clf, reg=train_oos_models(inventory_df=inventory_df,forecast_df=df)

/Users/ruhinshaikh/Desktop/Non-Pilot/retail_agentic/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:05:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📊 Classification Metrics:
Accuracy: 1.0
F1 Score: 1.0
ROC AUC: 1.0

📊 Regression Metrics:
RMSE: 1.1980857892736594
R2 Score: 0.987215687182324

✅ Models saved in models_forecast


In [122]:
inventory_df.columns

Index(['sku_id', 'product_name', 'product_category', 'current_stock',
       'max_capacity', 'cost_price', 'current_selling_price', 'store_id',
       'product_details', 'reorder_threshold', 'category', 'last_restock_date',
       'next_restock_date'],
      dtype='object')

In [123]:
df

,sale_date,sku_id,store_id,demand,lag_1,day_of_week,is_weekend,month,discount_percent,success_rate,propensity_score,date_added,days_predicted_for
0,2025-08-10,SKU0001,STORE01,6.59,2.000000,6.0,1.0,8.0,0.0,0.0,0.683727,2025-08-26 19:00:05.339593,7
1,2025-08-11,SKU0001,STORE01,6.32,6.591347,0.0,0.0,8.0,0.0,0.0,0.683727,2025-08-26 19:00:05.339593,7
2,2025-08-12,SKU0001,STORE01,5.40,6.317193,1.0,0.0,8.0,0.0,0.0,0.683727,2025-08-26 19:00:05.339593,7
3,2025-08-13,SKU0001,STORE01,3.73,5.402296,2.0,0.0,8.0,0.0,0.0,0.683727,2025-08-26 19:00:05.339593,7
4,2025-08-14,SKU0001,STORE01,3.63,3.730859,3.0,0.0,8.0,0.0,0.0,0.683727,2025-08-26 19:00:05.339593,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2025-08-12,SKU0100,STORE01,6.55,6.985379,1.0,0.0,8.0,0.0,0.0,0.211687,2025-08-26 19:00:05.339593,7
696,2025-08-13,SKU0100,STORE01,6.63,6.552753,2.0,0.0,8.0,0.0,0.0,0.211687,2025-08-26 19:00:05.339593,7
697,2025-08-14,SKU0100,STORE01,6.82,6.634023,3.0,0.0,8.0,0.0,0.0,0.211687,2025-08-26 19:00:05.339593,7
698,2025-08-15,SKU0100,STORE01,7.79,6.817474,4.0,0.0,8.0,0.0,0.0,0.211687,2025-08-26 19:00:05.339593,7


In [128]:
fdf=predict_oos_and_days_until_oos(inventory_df, df, clf, reg, horizon=7, sku_ids=None, store_ids=None)

In [129]:
fdf

,sku_id,store_id,current_stock,demand,predicted_oos,oos_probability,predicted_days_until_oos,forecast_date
0,SKU0001,STORE01,70,6.59,0,0.002092,7,2025-08-10
1,SKU0001,STORE01,70,6.32,0,0.002092,7,2025-08-11
2,SKU0001,STORE01,70,5.40,0,0.002092,7,2025-08-12
3,SKU0001,STORE01,70,3.73,0,0.002092,7,2025-08-13
4,SKU0001,STORE01,70,3.63,0,0.002092,7,2025-08-14
...,...,...,...,...,...,...,...,...
695,SKU0100,STORE01,178,6.55,0,0.001909,7,2025-08-12
696,SKU0100,STORE01,178,6.63,0,0.001909,7,2025-08-13
697,SKU0100,STORE01,178,6.82,0,0.001909,7,2025-08-14
698,SKU0100,STORE01,178,7.79,0,0.001909,7,2025-08-15


In [126]:
fdf.to_csv("data/test_oos.csv")